<a href="https://colab.research.google.com/github/KauaHenSilva/python_nn_breast_cancer/blob/main/breast_cancer_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recuperar arquivos

In [1]:
import os
tipo_de_recuperacao = "Ambos" #@param ["Download", "Drive", "Ambos"]
def repostirio_para_download():
  if os.path.exists('python_download_file_drive'):
    !cd python_download_file_drive && git pull
  else:
    !git clone https://github.com/KauaHenSilva/python_download_file_drive
  %pip install --upgrade --quiet -r python_download_file_drive/requirements.txt

## Recuperação por download

In [2]:
if tipo_de_recuperacao in ["Download", "Ambos"]:
  from google.colab import auth
  from googleapiclient.discovery import build
  auth.authenticate_user()
  repostirio_para_download()
  !python "python_download_file_drive/main.py" "https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link" arquivos

Cloning into 'python_download_file_drive'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 57 (delta 29), reused 29 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 13.51 KiB | 3.38 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Baixando arquivos: 100% 2/2 [00:09<00:00,  4.58s/arquivo]


## Recuperação por drive

In [3]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
# @markdown > https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/tmp_name" #@param {type:"string"}

In [4]:
if tipo_de_recuperacao in ["Drive", "Ambos"]:
  from google.colab import drive
  drive.mount('/content/drive')
  if not os.path.exists(path_atalho_drive):
    raise Exception("O caminho especificado não existe no drive.")

Mounted at /content/drive


## Referenciar arquivo no drive ou download

In [5]:
def get_referencia_valida(path):
    path_download = os.path.join("/content/arquivos", path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download


# breast cancer tuning

In [6]:
!pip install tensorflow==2.16.1 scikit-learn==1.5.0  scikeras==0.13.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.16.1 which is incompatible.


In [7]:
import scikeras
import pandas as pd
import tensorflow as tf
import sklearn

In [8]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.16.1', '1.5.0', '0.13.0')

In [9]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k
from sklearn.model_selection import GridSearchCV

In [10]:
path_entrada = get_referencia_valida("entradas_breast.csv")
path_saida = get_referencia_valida("saidas_breast.csv")

In [11]:
x = pd.read_csv(path_entrada)
y = pd.read_csv(path_saida)
x.shape, y.shape

((569, 30), (569, 1))

In [25]:
def criar_rede_drop(optimizer, loss, kernel_initializer, activation, neurons):
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
  return rede_neural

In [26]:
rede_neural = KerasClassifier(model = criar_rede_drop)
# parametros = { # Leva mais de uma hora.
#     'batch_size': [10, 30],
#     'epochs': [50, 100],
#     'model_optimizer': ['adam', 'sgd'],
#     'model_loss': ['binary_crossentropy', 'hinge'],
#     'model_kernel_initializer': ['random_uniform', 'normal'],
#     'model_activation': ['relu', 'tanh'],
#     'model_neurons': [16, 8]
# }
parametros = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16]
}

In [ ]:
# 2 * 2 * 2 * 2 * 2* 2 * 2 * 5
2 * 1 * 1 * 1 * 2 * 1 * 1 * 5

In [27]:
grid_search = GridSearchCV(estimator=rede_neural, param_grid=parametros, scoring='accuracy', cv=5)
grid_search = grid_search.fit(x, y)

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.4548 - loss: 2.4412
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6511 - loss: 0.5862
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6501 - loss: 0.5934
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6777 - loss: 0.5408
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7334 - loss: 0.4640
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7491 - loss: 0.5013
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7310 - loss: 0.4756
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8400 - loss: 0.3919
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8299 - loss: 0.3901
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8689 - loss: 0.3296
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9036 - loss: 0.31

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_accuracy: 0.4915 - loss: 2.6852   
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6220 - loss: 0.6822 
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6463 - loss: 0.5406 
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6499 - loss: 0.5546 
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6455 - loss: 0.5178 
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6917 - loss: 0.4920 
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7242 - loss: 0.5299 
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7380 - loss: 0.4917 
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7935 - loss: 0.4313 
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.7718 - loss: 0.4899 
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 

In [30]:
grid_search

GridSearchCV(cv=5,
             estimator=KerasClassifier(model=<function criar_rede_drop at 0x7e85f4ade340>),
             param_grid={'batch_size': [10, 30], 'epochs': [50],
                         'model__activation': ['relu'],
                         'model__kernel_initializer': ['random_uniform',
                                                       'normal'],
                         'model__loss': ['binary_crossentropy'],
                         'model__neurons': [16], 'model__optimizer': ['adam']},
             scoring='accuracy')

In [31]:
 melhores_parametros = grid_search.best_params_
 melhores_parametros

{'batch_size': 10,
 'epochs': 50,
 'model__activation': 'relu',
 'model__kernel_initializer': 'random_uniform',
 'model__loss': 'binary_crossentropy',
 'model__neurons': 16,
 'model__optimizer': 'adam'}

In [32]:
 melhor_precisao = grid_search.best_score_
 melhor_precisao

0.8981058841794752